# Zero-coupon vs. Market yields

A zero-coupon yeild is a "synthetic" construct. It is next to impossible to find an actual zero-coupon bond with the exact tenor that trades at those yeilds. It is primarily used as an input to quant/risk models because it is "pure." 

Market yields are shaped mostly by supply and demand. So it is a useful measure for a practical risk-free rate. 

pluto has [zero-coupon](https://plutopy.readthedocs.io/en/latest/YieldCurve.html#plutoPy.model.YieldCurve.IndiaZeroCoupon) rates starting from 2011-01-18 and a market [based bond index](https://plutopy.readthedocs.io/en/latest/Indices.html#plutoPy.model.Indices.IndiaGsecTimeSeries) starting from 2003-12-31. We will plot the yields of the two below to get a sense of the difference between the two.

In [ ]:
library(tidyverse)
library(ggthemes)
library(reshape2)
library(odbc)
library(plutoR)
library(quantmod)
library(lubridate)
library(ggrepel)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

#initialize
indices <- Indices() #for IndiaGsecTimeSeries
ycurves <- YieldCurve() #for IndiaZeroCoupon

In [ ]:
startEndDatesIndex <- (indices$IndiaGsecTimeSeries() %>%
    summarize(MAX_TS = max(TIME_STAMP), MIN_TS = min(TIME_STAMP)) %>%
    collect())

startEndDatesZero <- (ycurves$IndiaZeroCoupon() %>%
    summarize(MAX_TS = max(TIME_STAMP), MIN_TS = min(TIME_STAMP)) %>%
    collect())

startDate <- max(startEndDatesIndex$MIN_TS[[1]], startEndDatesZero$MIN_TS[[1]])
endDate <- min(startEndDatesIndex$MAX_TS[[1]], startEndDatesZero$MAX_TS[[1]])

indexDt <- (indices$IndiaGsecTimeSeries() %>%
            filter(TIME_STAMP >= startDate & TIME_STAMP <= endDate) %>%
            select(TIME_STAMP, NAME, YIELD = YTM) %>%
            collect() %>%
            mutate(MATURITY = as.numeric(strsplit(NAME, "_")[[1]][2])) %>% 
            select(TIME_STAMP, MATURITY, YIELD))

ycDt <- (ycurves$IndiaZeroCoupon() %>%
            filter(TIME_STAMP >= startDate & TIME_STAMP <= endDate 
                   & MATURITY %in% c(5, 10, 15, 20, 30)) %>%
            collect())

print("bond index data:")
print(head(indexDt))
print(tail(indexDt))

print("zero-coupon data:")
print(head(ycDt))
print(tail(ycDt))


In [ ]:
indexDt %>% 
    inner_join(ycDt, by=c('TIME_STAMP', 'MATURITY')) %>%
    print()

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)